In [1]:
# Created by Balakrishna Vagvala and Sherrill Kirk R
# This code depends upon the packages and libraries listed in the next cell
# Date of creation: 12/1/2020
# Script purpose: this script processes the OSU CCAL Lab delivered data, compares with expected fields file and make ready to be uploaded to database.

In [2]:
# We use openpyxl to load excel work book, this will help in having our current data as new sheet in the workbook
# we use shutil to copy a file from it's location to a desired location,
# this will help in copy pasting our original file to make changes on it
# We use difflib library and import sequencematcher function, this will help us in comparing strings and give us a match ratio
# We use numpy library and import it as np to make numerical operations for our program
# we use pandas library and immport it as np to make data operations for our lab data
# we use time library to import curent time of our function
# we use OS library for different functions such as writing data to file and saving files
# We use glob library and its glob function finds all the pathnames matching a specified pattern

In [3]:
#importing pandas for data operations, numpy for numerical operations. time is used to have present time.
import pandas as pd
import numpy as np
import time
#sequence matcher is used to compare strings and openpyxl is used to write workbooks.
from difflib import SequenceMatcher
from openpyxl import load_workbook
from shutil import copyfile
#importing reading functions and declaring the location of our file.
import os, glob

In [4]:
# mounting google drive for google collab, not needed for native IDE's
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
#Input Parameters
#Output path, where you want to save
currentoutputpath= r'D:\ROMN\working\Misc\4People\4Bala\WaterQualityProcessing\CCAL\2020\Preprocessed'

#Path to directory with files to be processed
path = r'D:\ROMN\working\Misc\4People\4Bala\WaterQualityProcessing\CCAL\2020'

#Wildcard syntax for files to be processed 
fileWildCard = "*.xlsx"

#File with expected fields in the lab delirable
fileCrossWalk = r'D:\ROMN\working\Misc\4People\4Bala\WaterQualityProcessing\CCAL\OSU_CCAL_ExpectedFields_CrossWalk_20201214.xlsx'

#Suffix name for output logfile and combined/appended file.
outputName = "OSUData2020"

In [6]:
# runtime for file
timestr = time.strftime("_%Y%m%d_%H%M")
runtimenow = timestr

# the opening the log file
logfile= open(currentoutputpath+"\\" + outputName + "_logifle" + runtimenow + ".txt","w+")

listofdataframes = []

all_files = glob.glob(os.path.join(path, fileWildCard))
for  no,f in enumerate(all_files):
    df = pd.read_excel(f, sheet_name = 1)
    #file to check crosswalks, i.e field names, this is an excel file's path
    #toread crosswalk file
    crosswalk = pd.read_excel(fileCrossWalk)
    #file to process , this is an excel file's path
#     data = "D:\Bala\OSU CCAL\input\GRKO_032620_changed_names.xlsx"

    
    
    filename= os.path.basename(f)
    print(filename)
    sep = '.'
    filename = filename.split(sep, 1)[0]

    

    firstline=[]
    firstline.append("File Being Processed is: " + filename+ "\n \n \n#########\n\n" )
    #toread the lab file, we mention sheet name of excel file in the second argument, here GRKO Data is our sheet name in excel file by lab
    # osudata = pd.read_excel(data, 'GRKO Data')
    #to specify sheet number instead of sheet name
#     osudata = pd.read_excel(data, sheet_name = 1)

    #here we process our data so that we have just the field names and field values in our dataframe
    osudata = df
    df = df.iloc[2:]
    df = df.rename(columns=df.iloc[0])
    df.reset_index(inplace=True)
    df = df.iloc[1:,1:]
    display(df)

    # to remove \n and \r from lab delivareables, they are influencing our data and output, so we are removing them

    dfStrip = osudata.replace('\n',' ', regex=True)

    dfStrip2 = dfStrip.replace('\r','', regex=True)

    df = dfStrip2

    df = df.iloc[2:]

    df = df.rename(columns=df.iloc[0])

    df.reset_index(inplace=True)

    df = df.iloc[1:,1:]

    osufields = list(df.columns)
    labfields = crosswalk['NativeDeliveryFIelds'].tolist()
    expectedfields = crosswalk['CrossWalkedFields'].tolist()
    # this cell is to check for coloumns and export irregularities into warning list
    
    matchingfields =[]
    lenghtoffields = []
    morethanninety = []
    lessthanninety = []
    newfields=[]
    duplicatefields=[]
    missedfields = []
    p=''
    q=''
    x=''
    if len(osufields)>len(labfields):
        lenghtoffields.append('\n\nThis '+filename+' file has new columns.\n')

    if len(osufields)<len(labfields):
        lenghtoffields.append('This '+filename+' file has less columns than expected.\n')

    duplicatefields.append ('\n\n#####\n\nFields with duplicate field names: \n') 
    missedfields.append ('\n\n######\n\nFields which are expected, but not in our data:\n')
    matchingfields.append ('\n\n#####\n\nFields that matched as expected:\n') 
    morethanninety.append ('\n\n#####\n\nFields without 100% match but had more than 90% Match:\n') 
    lessthanninety.append ('\n\n#####\n\nFields with less than 90% match but had more than 80% Match:\n') 
    newfields.append ('\n\n#####\n\nNew Fields these are not found in crosswalk and have been removed from data:\n') 

    for i in osufields:
        if i == 'Date': 
            x = osufields.index(i)
            osufields[x] = osufields[x-1] +(" ") +osufields[x]
            continue 
    for j in labfields:
        if j == 'Date':
            x = labfields.index(j)
            labfields[x] = labfields[x-1] +(" ") + labfields[x]
    df.columns = osufields

    #Bala change this code section to match the duplicate check you are performing in the Test America Script
    #I believe you are planning to throw an error if a field occurs more than once in the deliverable (date fields excluded)
    for i in osufields:   
        for j in labfields:
            if j != 'Date' and i != 'Date' :
                if 'Date' not in i:
                    if j in i:                
                        ratio = SequenceMatcher(None,i,j).ratio()
                        if ratio != 1:

                            duplicatefields.append("Warning: "+i+ " is a duplicate field, almost matched with "+i+" \n")                    

                            x = list(df.columns).index(i)
                            cols = [i for i in range(df.shape[1])]
                            cols.remove(x)
                            cols.remove(x+1)
                            df = df.iloc[:,cols]
                            del osufields[x]
                            del osufields[x]

    for i in osufields:     
        if i in labfields:
            matchingfields.append("column: "+ i +" has sucessfully matched \n")
            x = osufields.index(i)
            y = labfields.index(i)        
            osufields[x] = expectedfields[y]
            continue      

        else:
            result = ""
            count = 0
            ratio = 0  

            for words in labfields:
                ratio = SequenceMatcher(None,i,words).ratio()
                if ratio > count:
                    count = ratio
                    result =words         

            if count > 0.9:
                x = osufields.index(i)
                y = labfields.index(result)        
                morethanninety.append("Warning: "+osufields[x]+" was crosswalked to "+ result + " has matched with more than 90% accuracy \n")
                osufields[x] = expectedfields[y]



            elif count> 0.8:
                x = osufields.index(i)
                y = labfields.index(result)
                lessthanninety.append("Warning: "+osufields[x]+" was crosswalked to "+ result + " has matched with more than 80% accuracy  \n")

                osufields[x] = expectedfields[y]



    # temp to check the missing columns in our processed lab deliverable 
    missedfields = []
    missedfields.append ('\n\n######\n\nFields which are expected, but not in our data\n')
    temp =  expectedfields.copy()
    for i in osufields:
        if i in temp:
            temp.remove(i) 
    for i in temp:
        missedfields.append("column: "+ i + " is expected but is not in our current data \n ")        

    df.columns = osufields

    for i in osufields:
        if i not in expectedfields:
            newfields.append("Warning: "+i+" is a new field, not being processed\n")
            x = list(df.columns).index(i)
            del osufields[x]
            cols = [i for i in range(df.shape[1])]
            cols.remove(x)
            df = df.iloc[:,cols]


    df.columns = osufields
    # to reindex columns based on expected fields
    df = df.reindex(expectedfields, axis=1)
    # this will drop columns with no values which are picked up from re-indexing
    df = df.dropna(axis=1)

    worked_files = firstline  + lenghtoffields+ morethanninety +lessthanninety + duplicatefields + missedfields+newfields + matchingfields
    for i in range(len(worked_files)):
        logfile.write(worked_files[i])

    logfile.write('\n\n######\nthis is end of the Log File for '+filename+'\n\n###########\n\n')
          

#     pd.set_option("display.max_rows", None, "display.max_columns", None)
#     df

    #saving our new csv file into a downloadable format. and it can be saved
    # df.to_csv( "data_fieldsprocessed"+timestr+".csv", index=False, encoding='utf-8-sig')
    #if you want an excel output
    # f = open
    # df.to_excel("pre-processed"+timestr+".xls", index=False, encoding='utf-8-sig')

    # to make a duplicate of our file and export our processed 
    # copyfile ('D:\Bala\OSU CCAL\input\GRKO_032620.xlsx' , './processed'+filename+'.xlsx')
    processed_path = (currentoutputpath+"\\"+filename+'_processed.xlsx')
    copyfile (f , processed_path)
#     data

    # rename our copied file and place it at the location we want

    # os.rename(r'D:\Bala\OSU CCAL\output\processed.xlsx', r'D:\Bala\OSU CCAL\output\OSUCCAL_processed_' + filename + '.xlsx')

    # enter the path of the processed named copy of the original file
    # processed_path = 'D:\Bala\OSU CCAL\output\OSUCCAL_processed__filename.xlsx'
    # to load our data frame into excel sheet
    book = load_workbook(processed_path)
    writer = pd.ExcelWriter(processed_path, engine = 'openpyxl')
    writer.book = book
    # enter the desired sheet name below
    sheetname = 'preprocessed' #enter your sheet name here in the space, we can have it as a prefix as you want
    df.to_excel(writer, sheet_name = sheetname)
    writer.save()
    writer.close()
    listofdataframes.append(df)

logfile.close() 

GRKO_032620.xlsx


,SampleName,Project Code,Lab Number,Site ID,Remark,Delivery Date,Alkalinity\n(mg CaCO3/L),Date,Suspended Sediment \r(mg/L),Date,...,Cl\r\n(mg/L),Date,SO4-S\r\n(mg S/L),Date,Na\r\n(mg/L),Date,NO3\r\n(mg/L),Date,Duplicate NO3\r\n(mg/L),Date
1,GRKO101,GRKO,101,19NPS0000602,GRKO_S01 3/24/20 13:45,3/26/2020,134.90,4/28/2020,12.50,4/14/2020,...,11.66,5/13/2020,34.91,5/13/2020,17.07,6/2/2020,0.81,4/29/2020,0.82,4/29/2020


GRKO_042820.xlsx


,SampleName,Project Code,Lab Number,Site ID,Remark,Delivery Date,Alkalinity (mg CaCO3/L),Date,Suspended Sediment (mg/L),Date,...,Cl\r\n(mg/L),Date,SO4-S\r\n(mg S/L),Date,Duplicate SO4-S\r\n(mg S/L),Date,Na\r\n(mg/L),Date,NO3\r\n(mg/L),Date
1,GRKO102,GRKO,102,20NPS0000650,4/22/20 13:15,4/28/2020,145.33,4/29/2020,17.97,5/7/2020,...,12.61,5/20/2020,36.43,5/20/2020,36.60,5/20/2020,17.53,6/2/2020,0.41,5/13/2020


GRKO_050820_060820.xlsx


,SampleName,Project Code,Lab Number,Site ID,Remark,Delivery Date,Alkalinity (mg CaCO3/L),Date,Suspended Sediment (mg/L),Date,...,SO4-S\r\n(mg S/L),Date,Duplicate SO4-S\r\n(mg S/L),Date,Na\r\n(mg/L),Date,NO3\r\n(mg/L),Date,Duplicate NO3\r\n(mg/L),Date
1,GRKO103,GRKO,103,20NPS0000651,"GRKO_S01, 5/5/2020, 12:30",5/8/2020,122.01,5/11/2020,34.10,5/19/2020,...,33.68,7/9/2020,33.67,7/9/2020,15.80,6/29/2020,0.68,6/11/2020,NaN,NaN
2,GRKO104,GRKO,104,20NPS0000652,GRKO_S01 5/19/2020 10:30,5/22/2020,116.54,5/27/2020,73.97,6/3/2020,...,22.96,7/9/2020,NaN,NaN,11.92,6/29/2020,0.52,6/11/2020,NaN,NaN
3,GRKO105,GRKO,105,20NPS0000653,GRKO_S01 6/2/2020 1:15,6/8/2020,74.80,6/12/2020,54.17,6/17/2020,...,14.64,7/9/2020,NaN,NaN,7.54,6/29/2020,0.29,6/11/2020,0.29,6/11/2020


GRKO_071620.xlsx


,SampleName,Project Code,Lab Number,Site ID,Remark,Delivery Date,pH,Date,Alkalinity (mg CaCO3/L),Date,...,UTP\r\n(mg P/L),Date,Cl\r\n(mg/L),Date,SO4-S\r\n(mg S/L),Date,Na\r\n(mg/L),Date,NO3\r\n(mg/L),Date
1,GRKO106,GRKO,106,20NPS0000654,"grko_s01, 7/14/2020 1:10",7/16/2020,8.2,7/23/2020,111.31,7/23/2020,...,0.038,10/22/2020,5.12,8/13/2020,18.84,8/13/2020,10.22,8/3/2020,0.09,7/30/2020


ROMN_112219_121719.xlsx


,SampleName,Project Code,Lab Number,Site ID,Remark,Delivery Date,Alkalinity (mg CaCO3/L),Date,Duplicate Alkalinity (mg CaCO3/L),Date,...,Cl \n(mg/L),Date,SO4-S \n(mg S/L),Date,Na \n(mg/L),Date,Duplicate Na \n(mg/L),Date,NO3 (mg/L),Date
1,ROMN237,ROMN,237,18NPS0000521,GRKO_S01 11/14/19 14:30,11/22/2019,152.06,11/25/2019,151.86,11/25/2019,...,10.20,3/12/2020,28.53,3/12/2020,15.89,2/12/2020,16.01,2/12/2020,0.63,2/12/2020
2,ROMN238,ROMN,238,19NPS0000601,GRKO_S01 12/13/19 10:45,12/17/2019,150.85,12/17/2019,152.07,12/17/2019,...,10.01,3/12/2020,32.30,3/12/2020,16.25,2/12/2020,NaN,NaN,0.99,2/12/2020


In [7]:
#Combine all Individual Pre-processesd files into the final appended/merged .csv file
combined_file = pd.concat( listofdataframes) 
combined_file.to_csv( currentoutputpath+"\\PreProcessedAppended_"+ outputName + "_" + timestr + ".csv", index=False, encoding='utf-8-sig')

In [8]:
filename

'ROMN_112219_121719'